<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<font color="#138D75">**WEkEO Training Service**</font> <br>
**Copyright:** 2023 EUMETSAT <br>
**License:** MIT <br>
**Authors:** Anna-Lena Erdmann (EUMETSAT)

<div class="alert alert-block alert-success">
<h3>xcube Viewer: Create and Visualize a Data Cube using WEkEO and the xcube Viewer</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
  - **<a href="https://my.wekeo.eu/user-registration" target="_blank">A WEkEO account</a>** if you are using or plan to use WEkEO.
  - access and execution of this notebook inside the **<a href="https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwi3tvDwzdqFAxWCh_0HHSo1DwEQFnoECA8QAQ&url=https%3A%2F%2Fjupyterhub.prod.wekeo2.eu%2F&usg=AOvVaw1qgpIG2_El9SA6di0OfwQ6&opi=89978449" target="_blank"> WEkEO JupyterHub</a>**
  - some knowledge about the data access through the **<a href="https://github.com/wekeo/wekeo4data/blob/main/wekeo-climetlab/01_WEkEOCliMetLab_Introduction.ipynb" target="_blank">CliMetLab WEkEO Plugins</a>**

</div>
<hr>

# 1 Create and Visualize a Data Cube using WEkEO and the xcube Viewer

### Data used

| Product Description  | WEkEO HDA ID | WEkEO metadata | CliMetLab ID
|:--------------------:|:-------------:|:-----------------:|:-----------------:|
| ERA5 hourly data on single levels from 1940 to present  | EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AECMWF%3ADAT%3AREANALYSIS_ERA5_SINGLE_LEVELS" target="_blank">link</a> | wekeo-ecmwf-reanalysis-era5-single-levels |

### Learning outcomes

At the end of this notebook you will know;

* how to create xcube-compatible data cubes from WEkEO data 
* how to correctly write a config file for xcube Viewer
* how to visualize and interact with data in the xcube viewer

### Outline

The <a href="https://xcube.readthedocs.io/en/latest/index.html" target="_blank">xcube</a> software package has been developed to generate, manipulate, analyse, and publish data cubes from Earth Observation data. This notebook shows how xcube, especially the xcube viewer can be used with EO data coming from WEkEO.

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Import](#section0)
 2. [Download Datasets with the CliMetLab Plugin](#section1)
 3. [Creating a Datacube with xArray](#section2)
 4. [Creating a config file](#section3)
 5. [Viewing the Data with xcube Viewer](#section4)

<hr>

<div class="alert alert-info" role="alert">

## 1. <a id='section0'></a>Import
[Back to top](#TOC_TOP)
    
</div>

In [ ]:

import os                                       # a library that allows us access to basic operating system commands like making directories    
import climetlab as cml                         # a library that gives us access to WEkEO datasets  
from climetlab_wekeo_datasets import hda2cml      # a library that converts the WEkEO HDA syntac to climetlab syntax
import xarray as xr                             # a library that supports the use of multi-dimensional arrays in Python
from xcube.webapi.viewer import Viewer          # a library to visualize datacubes inside the Jupyter Notebook 


Set the xcube Viewer url to show the xcube Viewer inside the notebook. 

**Note: Replace the `<your_username>` with your WEkEO username.**

In [2]:
os.environ["XCUBE_JUPYTER_LAB_URL"] = "https://jupyterhub.prod.wekeo2.eu/user/<your_username>/"

<div class="alert alert-info" role="alert">

## 2. <a id='section1'></a>Download Datasets using the CliMetLab Plugin
[Back to top](#TOC_TOP)
    
</div>

In this notebook we will work with the **ERA5 Air temperature reanalysis**. You can get more information on this dataset and explore it <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AECMWF%3ADAT%3AREANALYSIS_ERA5_SINGLE_LEVELS" target="_blank">here</a>. 

The API request is derived from the WEkEO Viewer. 

We will download and transform the data to xarray using the CliMetLab WEkEO Plugin.

When using the climetlab functions the first time, you will have to enter WEkEO username and password before the data download.  

In [ ]:
era5_request = {
  "dataset_id": "EO:ECMWF:DAT:REANALYSIS_ERA5_SINGLE_LEVELS",
  "product_type": [
    "reanalysis"
  ],
  "variable": [
    "2m_temperature"
  ],
  "year": "2023",
  "month": [
    "01"
  ],
  "day": [
    "01"
  ],
  "time": [  "00:00",  "01:00",  "02:00",  "03:00",  "04:00",  "05:00",  
  "06:00",  "07:00",  "08:00",  "09:00",  "10:00",  "11:00",  "12:00", 
  "13:00",  "14:00",  "15:00",  "16:00",  "17:00",  "18:00",  "19:00",  
  "20:00",  "21:00",  "22:00",  "23:00"
  ],
  "format": "netcdf"
}

In [ ]:
# download dataset and convert it to xarray

ds_id, args = hda2cml(era5_request)
era5_cmlds= cml.load_dataset(ds_id, **args)
era5_ds = era5_cmlds.to_xarray()

In [12]:
era5_ds

<xarray.Dataset> Size: 199MB
Dimensions:    (longitude: 1440, latitude: 721, time: 24)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 192B 2023-01-01 ... 2023-01-01T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float64 199MB dask.array<chunksize=(24, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-09 07:29:35 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

<div class="alert alert-info" role="alert">

## 3. <a id='section2'></a>Creating a Datacube with xArray
[Back to top](#TOC_TOP)
    
</div>

From the xarray overview we see, that the longitude is in the range of `[0, 360]`. FOr compatibility reasons we convert it to the range of `[-180, 180]`. 

In [13]:
era5_ds.coords['longitude'] = (era5_ds.coords['longitude'] + 180) % 360 - 180
era5_ds_reshaped = era5_ds.sortby(era5_ds.longitude)

Subset the global dataset to our area of interest os Spain:

In [14]:
spain_era5 = era5_ds_reshaped.sel( latitude=slice(44.1, 35.6), longitude = slice(-10, 4))[['t2m']]
spain_era5.attrs["title"] = "Air Temp Spain 2023-01-01"

<hr>
<div class="alert alert-info" role="alert">

## 4. <a id='section3'></a>Creating a config file
[Back to top](#TOC_TOP)

</div>


To add data cubes to the xcube viewer, they have to be defined in a config file.

Two main parts of the config file is the ``Datasets`` description and the ``Styles`` description. As our datacube is already in our environment, we don't have to explicitly define the ``Datasets`` parameters it in the config file. So we are only setting the ``Styles`` parameters. 

The ``Style`` description contains:
* **Identifier** *unique name for the style*
* **ColorMappings** 
    * **VariableName** *must be identical to the variable name in the cube*
        * **ColorBar** *name of colorbar as in matplotlib*
        * **ValueRange** *range of the dataset*

In [15]:
viewer = Viewer(server_config={
    "Styles": [
        {
            "Identifier": "temperature",
            "ColorMappings": {
                "t2m": {
                    "ValueRange": [250.0, 300.0],
                    "ColorBar": "inferno"
                },
            }
        }
    ]
})

Next, we add our dataset to the Viewer

In [9]:
viewer.add_dataset(spain_era5, style="temperature")

'57201e6c-a6ba-4e53-bee0-da9c95d0a84b'

<hr>
<div class="alert alert-info" role="alert">

## 5. <a id='section4'></a>Viewing the Data with xcube Viewer
[Back to top](#TOC_TOP)

</div>


The `viewer.show()` function shows an instance of the xcube Viewer inside your norebook. 

You can now explore the datacubes by <font color="#138D75"> browse through time, create a time series at one point, compare different locations ...</font>

In [ ]:
viewer.show()

<img src='./img/01_xcubeviewer_show.png' align='left' width='80%'/>